# Supervised classification
## Problem

Write a classifier for the FashionMNIST dataset consisting of 28x28 grayscale images of fashion accessories.

Classifiers to use:
1. SVM (Support Vector Machine) with linear, polynomial of degree 2, and RBF kernels;
2. Random forests with varying number of trees;
3. k-NN (k Nearest Neighbors) with varying k;
4. Multi-Layer Perceptron (MLP) with varying number and size of hidden layers.

For each classifier 10 folds cross validation (CV) should be used.

## SVM

For SVM implementation scikit-learn library was used. Some of used classes:

In [5]:
from sklearn.svm import SVC  # for polynomial and RBF kernels
from sklearn.svm import LinearSVC  # for linear kernel
from sklearn.preprocessing import StandardScaler  # scaling the images
from sklearn.decomposition import PCA  # for speeding up the training
from sklearn.pipeline import Pipeline  # for setting the classifier model
from sklearn.model_selection import GridSearchCV  # for cross validation

### Experimental setup

Linear kernel has only one parameter C which is how strictly SVM tries to avoid misclassification.

* Small C => wider margin, more tolerance for mistakes;
* Large C => narrow margin, tries to classify everything correctly.

However too large value of C can cause overfitting. So, the best value of C will be chosen after cross validation.

In [7]:
linear_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("pca", PCA(n_components=100)),
    ("svm", LinearSVC(dual=False, max_iter=10000))
])
linear_params = {"svm__C": [0.01, 0.1, 1, 10, 100]}

Polynomial kernel uses two more parameters (apart from C): $\gamma$ and coef0.

$\gamma$ controls how strongly two points influence each other:
* Large $\gamma$ => kernel focuses on points very close to each other => highly curved, very wiggly boundaries;
* Small $\gamma$ => smoother, more general boundary.

Coef0 controls how much the model uses interaction term (dot product) or constant term (bias):
* Small coef0 => model emphasizes the dot product;
* Large coef0 => model gives more weight to the constant term.

In [9]:
poly_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("pca", PCA(n_components=100)),
    ("svm", SVC(kernel="poly", degree=2))
])
poly_params = {
    "svm__C": [0.01, 0.1, 1, 10],  # less values then in linear case in order to speed up the training a bit
    "svm__coef0": [0, 0.5, 1],
    "svm__gamma": ["scale", "auto"]
}

RBF kernel uses only C and $\gamma$ parameters.

$\gamma$ controls how far the influence of a single training point spreads:
* Large $\gamma$ => influence only on nearby points => very wiggly and complex boundary;
* Small $\gamma$ => influence on a wider area => smooth boundary.

In [10]:
rbf_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("pca", PCA(n_components=100)),
    ("svm", SVC(kernel="rbf"))
])
rbf_params = {
    "svm__C": [0.1, 1, 10],
    "svm__gamma": ["scale", "auto"]
}

### Results

Linear kernel:
![image](./img/svm_linear.png)
Quadratic polynomial kernel:
![image](./img/svm_poly.png)
RBF kernel:
![image](./img/svm_rbf.png)

As we can see, test prediction time was increasing with every type of kernel because of the increasing complexity and amount of computations (linear kernel is the easiest and the most basic one, therefore needs much less time than others). However the time spent on cross validation in RBF kernel was less than in polynomial one most likely due to less amount of varying parameters values. If we add 4 more values in a list of values for C or gamma (making the total number of possible parameters values the same as in polynomial kernel), CV time in RBF would have also been significantly higher than in polynomial case.

In the same time the classification performance was getting better with every type of kernel rising from 0.8299 in linear to 0.8922 in RBF which justifies the reason to use these kernels instead of polynomial.

From classification report and confusion matrix it is also noticeable that the algorithm had the worst classification performance for the 6th class of images often confusing it with the 0th one and vice versa (which are shirt and T-shirt respectively).

## Random forests

For the model of random forest classifier RandomForestClassifier of scikit-learn was used (apart from some already mentioned classes of the same library).

### Experimental setup

Random forest classifiers use the number of decision trees as the parameter. Each tree sees a random subset of the training data, considers only a random subset of features (pixels) and gives its own prediction. The forest chooses the label with the most votes. So the more decision trees we have, the stronger final prediction will be (however too large amount of trees may result also in overfitting and increasing computational requirements).

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {
    "n_estimators": [10, 50, 100, 200, 300]  # numbers of trees
}

clf = RandomForestClassifier(
    max_depth=None,
    n_jobs=-1,
    random_state=42
)

# 10 folds cross validation
grid = GridSearchCV(
    estimator=clf,
    param_grid=param_grid,
    cv=10,
    n_jobs=-1,
    verbose=1
)

### Results

![image](./img/forest.png)

As we can see, for random forests the time needed for cross validation and final training is less than for SVM with linear kernel while having higher classification performance than linear one.

This in general is due to the fact that random forest builds many simple decision trees each of which considers only part of the data and features and it is easy to parallelize because trees are built independently. While linear SVM solves one big expensive task and is limited for parallelization.

## K Nearest Neighbors (k-NN)

For the model of k-NN classifier KNeighborsClassifier of scikit-learn was used along with KFold to shuffle folds for cross validation (apart from some already mentioned classes of the same library).

In [13]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, KFold

### Experimental setup

K-NN is a method that looks at some most similar items it's seen before and chooses the majority label. So it uses k to determine how many neighbors to look at:
* Small k => sensitive to noise;
* Large k => smoother decisions.

In [14]:
param_grid = {
    "n_neighbors": [1, 3, 5, 7, 9, 11]  # different values of k
}

knn = KNeighborsClassifier()

# 10 folds cross validation
grid = GridSearchCV(
    estimator=knn,
    param_grid=param_grid,
    cv=KFold(n_splits=10, shuffle=True, random_state=42),  # shuffle folds, so that each fold contains a proper mix of classes
    scoring="accuracy",
    n_jobs=-1,
    verbose=1,
)

### Results

![image](./img/k-nn.png)

As we can see, in this case the time needed for cross validation and final training is significantly less than in all previous methods. While the classification performace is higher than in SVM with linear kernel but less than in others.

Such a good time performance is achieved by the fact that k-NN doesn't have any weights or complex equations, it just stores the dataset and compares.

As for classification performance, it may be lower than in some other methods because k-NN treats all input features as equally important, even if some of them are useless or misleading. So features that have nothing to do with the true class can still influence the distance calculation and therefore the prediction.

## Multi-Layer Perceptron

For MLP implementation pytorch library was used. Some of used classes:

In [16]:
import torch
import torch.nn as nn  # for different neural network methods
import torch.optim as optim  # for adjusting weights
from torch.utils.data import DataLoader, Subset  # for manipulations with dataset

### Experimental setup

In [17]:
DEVICE = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {DEVICE} device")

BATCH_SIZE = 128
EPOCHS = 10
LR = 1e-3  # learning rate
NUM_CLASSES = 10

Using mps device


Batch is a small chunk of data used for processing. During one epoch all batches are processed. Learning rate determines how much the weights change at each update.

As for the device, in my case it's mps which is a GPU for Apple Silicon devices.

In [18]:
from sklearn.model_selection import KFold

# varying number and size of hidden layers
param_grid = [
    {"hidden_layers": [128]},
    {"hidden_layers": [256]},
    {"hidden_layers": [128, 64]},
    {"hidden_layers": [256, 128]},
]

# set cross validation with 10 shuffled folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

Each layer in MLP takes simple patterns and combines them into more complex patterns.
* 1 hidden layer => simple patterns;
* 2 hidden layers => patterns of patterns (more expressive).

The size of hidden layers (number of neurons in them) controls how rich the internal representation is:
* Few neurons => simpler patterns;
* Many neurons => complex patterns.

### Results

Cross validation:

![image](./img/mlp_cv.png)

Final training and testing:

![image](./img/mlp_res.png)

As we can see, cross validation process in MLP is much slower than in k-NN, random forests and SVM with linear kernel but still faster than SVM with polynomial and RBF kernels. While its test accuracy is better than in those first 3 mentioned methods but worse than the rest of SVM kernels.

The time spent in MLP is still less than in some other methods due to linear growth of training time (instead of quadratic or worse in SVM), mini-batch training and GPU acceleration.

Test accuracy in case of MLP is worse than in SVM with polynomial and RBF kernels because in general it has many free parameters which makes it easier to underfit or overfit. SVMs are more strongly regularized by design and penalize complex boundaries automatically.

## Conclusion

Overall, SVM classifier with polynomial and RBF kernels seems to have the best test accuracy among all of these implemented methods. But in the same time it spends really huge amount of time on training and cross validation.

In terms of time performance k-NN classifier seems to be the best. However it has worse accuracy than most of the other methods.

So, in general, if the time is very important, it's better to use random forests as they spend much less time on training and CV than MLP or SVM and have sufficient accuracy.

In the opposite case it's better to use MLP because it has pretty good accuracy in comparison with most of the other methods and spend less time than SVM with polynomial or RBF kernel.

SVM with linear kernel is better to use on the dataset which is clearly linearly separable. Otherwise it has too low accuracy in comparison with other methods.